# DC Properties

In [3]:
import os
import mechanize
from bs4 import BeautifulSoup as soup
import json
import pandas as pd
import re


def clean_str(s):
    txt =  s.replace("\n"," ").replace("\t","").strip().encode("utf-8","ignore")
    txt = txt.replace("\r"," ")
    txt = re.sub(r'\xc2|\xa0',' ',txt)
    return txt

def reduce_spaces(s):
    return re.sub("[ ]{2,}"," ",s)

def saveWardSales(ward=1,page=1,verbose=False):
    last_page=999
    page_dfs=[]
    header = ["ssl","premise_address","owner_name","neighborhood","sub_neighborhood","use_code",
              "sale_price","recordation_date","total_assessment"]
    br = mechanize.Browser()    
    while page <= last_page:
        if verbose and page % 10==0: print "processing page %d" % page
        if page==1:
            br.open("https://www.taxpayerservicecenter.com/RP_Results.jsp?search_type=Sales")
            br.select_form(name="SearchForm")
            br["selectWard"]=[str(ward)]
            resp = br.submit()
            html = soup(resp.read(),"html.parser")

            # set the number of pages to crawl for this ward
            href=html.find("a",text="last").attrs['href']
            m=re.search(r"page=([0-9]*)",href)
            last_page = int(m.group(1))
        else:
            resp=br.open("https://www.taxpayerservicecenter.com/RP_Results.jsp?page=%d"%page)
            html = soup(resp.read(),"html.parser")    

        table = html.find("table",attrs={"class":"","border":"0","cellpadding":4,"cellspacing":2})
        i = 0
        rows=list()
        for tr in table.find_all("tr"):  
            if i>0:
                row = [clean_str(td.text) for td in tr.find_all("td")]
                values = row
                row = { e[0]:e[1] for e in zip(header,values) }                
                rows.append(row)
            i+=1
        page_dfs.append(pd.DataFrame(rows,dtype=str))
        page += 1

    ward_df = pd.concat(page_dfs,ignore_index=True)
    ward_df.to_csv("ward%d.csv" % (ward),index=False)
    return ward_df

def getDetails(ssl,br=None):
    if br is None:
        br = mechanize.Browser() 
        br.open("https://www.taxpayerservicecenter.com/RP_Results.jsp?search_type=Sales")
    resp = br.open("https://www.taxpayerservicecenter.com/RP_Detail.jsp?ssl=%s" % (ssl.replace(" ",'%20')))
    html= soup(resp.read(),"html.parser")
    tables = html.find_all("table",attrs={"cellspacing":"2"})
    items = {"ssl":ssl}
    if tables>0:
        for table in tables[:3]:    
            for tr in table.find_all("tr"):
                capture_next=False
                for td in tr.find_all("td"):
                    if not capture_next:
                        capture_next = not (re.search(":",td.text) is None)
                        label=td.text.replace(":","")
                    else:
                        value = reduce_spaces(clean_str(td.text))
                        value = value.replace("; "," ")
                        items.update({label:value})
                        capture_next=False
    return items

def getFeatures(ssl,br=None):
    if br is None:
        br = mechanize.Browser() 
        br.open("https://www.taxpayerservicecenter.com/RP_Results.jsp?search_type=Sales")
    resp = br.open("https://www.taxpayerservicecenter.com/weblogic/CAMA?ssl=%s" % (ssl.replace(" ",'%20')))
    html= soup(resp.read(),"html.parser")
    tables = html.find_all("table",attrs={"cellspacing":"2","cellpadding":"2","align":"left"})
    items = {"ssl":ssl}
    if tables>0:
        for table in tables[:3]:    
            for tr in table.find_all("tr"):
                capture_next=False
                for td in tr.find_all("td"):
                    if not capture_next:                        
                        label=td.text
                        capture_next=True
                    else:
                        value = reduce_spaces(clean_str(td.text))
                        value = value.replace("; "," ")
                        items.update({label:value})
                        capture_next=False
    return items

def mapSave(f,ssls,filename,chunk_size=100,resume=True):
    br = mechanize.Browser() 
    br.open("https://www.taxpayerservicecenter.com/RP_Results.jsp?search_type=Sales")
    dfs=list()
    if os.path.exists(filename) and resume:
        prev = pd.read_csv(filename)
        dfs.append(prev)
        ssls = list(set(ssls).difference(set(prev['ssl'])))
    i=0
    chunk = ssls[i*chunk_size:(i+1)*chunk_size]
    while len(chunk)>0:
        dfs.append(pd.DataFrame([f(c,br) for c in chunk]))
        pd.concat(dfs,ignore_index=True).to_csv(filename,index=False)
        i += 1
        chunk = ssls[i*chunk_size:(i+1)*chunk_size]

class Status(object):
    
    def __init__(self,iterable):
        self.n = float(len(iterable))
        self.i = 0
        
    def pprint(self,every=100):
        if self.i % every == 0: print "completed %02.4f%%" % (self.i/self.n)
        self.i += 1
        return True

    

In [66]:
# search by ward
sales = list()
for ward in range(1,9):
    print "processing ward %d" % ward
    sales.append(saveWardSales(ward=ward))



processing ward 1
processing ward 2
processing ward 3
processing ward 4
processing ward 5
processing ward 6
processing ward 7
processing ward 8


In [72]:
all_wards = pd.concat(sales,ignore_index=True)
all_wards.to_csv("sales.csv",index=False)

In [196]:
all_wards = pd.read_csv("sales.csv")
ssls=all_wards['ssl']
n=len(ssl)
i=0
br = mechanize.Browser() 
br.open("https://www.taxpayerservicecenter.com/RP_Results.jsp?search_type=Sales")
ssls = ssls[:50]
s = Status(ssls)
details = [ getDetails(ssl,br) for ssl in ssls if s.pprint(200) ]    

completed 0.00%


In [197]:
pd.DataFrame(details,dtype=str).to_csv("details.csv",index=False)

In [198]:
pd.DataFrame(details,dtype=str)['Mailing Address'][:5]

0           1701 V ST NW WASHINGTON DC20009-2610
1           PO BOX 34181 WASHINGTON DC20043-4181
2    1702 FLORIDA AVE NW WASHINGTON DC20009-2622
3           2119 R ST NW WASHINGTON DC20008-1908
4     912 ALABAMA AVE SE WASHINGTON DC20032-4247
Name: Mailing Address, dtype: object

In [4]:
all_wards = pd.read_csv("sales.csv")
ssls=all_wards['ssl']
mapSave(getFeatures,ssls,"features.csv")    

KeyboardInterrupt: 

In [ ]:
all_wards = pd.read_csv("sales.csv")
ssls=all_wards['ssl']
mapSave(getDetails,ssls,"details.csv")    